<a href="https://colab.research.google.com/github/ppug15510-jpg/Exercise_2025/blob/main/196fserver_197config.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#196 파일 전송 서버 만들기
import socketserver
from os.path import exists, getsize

HOST = ''
PORT = 9010
CHUNK_SIZE = 256*1024

class MyTcpHandler(socketserver.BaseRequestHandler):
   def handle(self):
      data_transferred = 0
      print('[%s] 연결됨' %self.client_address[0])
      filename = self.request.recv(1024)
      filename = filename.decode()

      if not exists(filename):
         return

      fsize = getsize(filename)
      data = f'#{fsize}#'
      self.request.send(data.encode())
      with open(filename, 'rb') as f:
         try:
            print(f'파일 {filename} / {fsize} 바이트 전송 시작...')
            data = f.read(CHUNK_SIZE)
            while data:
               data_transferred += self.request.send(data)
               data = f.read(CHUNK_SIZE)
         except Exception as e:
            print(e)

      print(f'전송완료 {filename}, 전송량 {data_transferred:,}/{fsize:,} 바이트')

def runServer():
   print('+++ 파일 서버를 시작합니다.')
   print('+++ 파일 서버를 끝내려면 Ctrl-C를 누르세요.')

   try:
      server = socketserver.TCPServer((HOST, PORT), MyTcpHandler)
      server.serve_forever()
   except KeyboardInterrupt:
      print('--- 파일 서버를 종료합니다.')

runServer()

+++ 파일 서버를 시작합니다.
+++ 파일 서버를 끝내려면 Ctrl-C를 누르세요.
--- 파일 서버를 종료합니다.


In [19]:
#197 파일 수신 클라이언트 만들기
{
"filename": "다운로드 할 이름",
"host": "127.0.0.1",
"port": "9010"
}

{'filename': '다운로드 할 이름', 'host': '127.0.0.1', 'port': '9010'}

In [20]:
from tkinter import *
import tkinter.ttk as ttk
from time import time
from threading import Event, Thread
import socket
import json

CHUNK_SIZE = 256*1024

In [21]:
class Downloader():
    def __init__(self):
        self.filename = ''
        self.filesize = 0
        self.host = ''
        self.port = 0

        #Tkinter 객체 생성 및 타이틀 설정
        self.root = Tk()
        self.root.title('다운로더')
        self.statusmsg = StringVar()

        #프레임 생성 및 배치
        content = ttk.Frame(self.root, padding=(6, 6, 6, 6))
        content.grid(column=0, row=0, sticky='nwes')

        #위젯 생성 및 배치
        self.pgress = ttk.Progressbar(content, orient=HORIZONTAL, length=400,
                        mode='determinate')
        startbtn = ttk.Button(content, text="다운로드 시작", command=self.start)
        self.status = ttk.Label(content, textvariable=self.statusmsg,
                        anchor='w')

        self.pgress.grid(column=0, row=0, pady=5, sticky='w')
        startbtn.grid(column=0, row=1, sticky='e')
        self.status.grid(column=0, row=2, sticky='w')

        #config.json 파일 읽음
        self.readConfig()

In [22]:
def readConfig(self):
        try:
            with open('config.json') as f:
                data = json.load(f)
        except Exception as e:
            print(str(e))
            data = {'filename': 'default.txt', 'host': '127.0.0.1', 'port': 9010} # 예외 처리 시 기본값 설정

        self.filename = data['filename']
        self.host = data['host']
        self.port = int(data['port'])
        self.statusmsg.set(f'다운로드할 파일:{self.filename}')

In [23]:
class Downloader():
    ...

    def start(self):
        Thread(target=self.startDownload).start()

In [24]:
class Downloader():
    ...

    def startDownload(selfself):
        stime = time()
        count = 0
        drate = 0
        data_transferred = 0
        self.pgress['value'] = 0

        HOST = self.host
        PORT = self.port

        try:
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
                sock.connect((HOST, PORT))
                sock.sendall(self.filename.encode())

                data = sock.recv(1024)
                if not data:
                    self.statusmsg.set('서버에 파일이 존재하지 않습니다.')
                    return

                if data.decode()[0] == '#':
                    self.filesize = int(data.decode().split('#')[1])

                self.pgress['maximum'] = self.filesize

                data = sock.recv(CHUNK_SIZE)
                with open(self.filename, 'wb') as f:
                    try:
                        while data:
                            f.write(data)
                            data_transferred += len(data)
                            self.pgress['value'] = data_transferred
                            count += 1
                            if count % 256 == 0:
                                t = (time() - stime)
                                if t != 0:
                                    drate = data_transferred / t
                                    drate /= 1024
                                msg1 = f'파일크기:{self.filesize:,}'
                                msg2 = f'받은양:{data_transferred:,}'
                                msg3 = f'전송속도:{int(drate):,}KB/s'
                                msg = f'{msg1} {msg2} {msg3}'
                                self.statusmsg.set(msg)
                            data = sock.recv(CHUNK_SIZE)
                    except Exception as e:
                        self.statusmsg.set(str(e))
                        return

                self.statusmsg.set(f'{self.filename} 전송완료!!')
        except Exception as e:
            self.statusmsg.set(str(e))

In [25]:
class Downloader():
    ...

    def run(self):
        self.root.mainloop()

In [26]:
if __name__ == '__main__':
    downloaderGUI = Downloader()
    downloaderGUI.run()

AttributeError: 'Downloader' object has no attribute 'root'

In [27]:
{
  "filename": "TheManWhoLaugh.mp4",
  "host": "127.0.0.1",
  "port": "9010"
}

{'filename': 'TheManWhoLaugh.mp4', 'host': '127.0.0.1', 'port': '9010'}